In [ ]:
class Number:
    def __init__(self, value):
        self.value = value

    @property
    def magnitude(self):
        return self.value
    
    def add(self, number, end="not_used"):
        # Same signature as Pair, with end=
        self.value += number.value
    
    def __ge__(self, other):
        return self.value >= other.value
    
    def split(self):
        floor, mod = divmod(self.value, 2)
        return Number(floor), Number(floor + bool(mod))
    
    def __repr__(self):
        return str(self.value)

In [ ]:
class Pair:
    def __init__(self, left, right):
        
        def number_or_pair(item):
            if isinstance(item, int):
                return Number(item)
            if isinstance(item, Number):
                return Number(item.value)
            return Pair(*item)
        
        self.left = number_or_pair(left)
        self.right = number_or_pair(right)
    
    @classmethod
    def from_string(cls, string):
        return cls(*eval(string))
    
    def __repr__(self):
        return str([self.left, self.right])
    
    def __iter__(self):
        return iter((self.left, self.right))
    
    def __add__(self, other):
        return Pair(self, other)

    def add(self, number, end):
        # Add number to the far-left or far-right number, end="left" or end="right"
        item = getattr(self, end)
        while not isinstance(item, Number):
            item = getattr(item, end)
        item.add(number)

    @property
    def magnitude(self):
        return 3*self.left.magnitude + 2*self.right.magnitude

In [ ]:
def explode_and_split(pair):

    def explode(pair, left=None, right=None, level=0):
        nonlocal changed
        # print(f"    {pair=}, {left=}, {right=}, {level=}, {changed=}")

        def pair_add(pair, number, end):
            if pair is not None:
                pair.add(number, end=end)
            
        if (not changed) and isinstance(pair.left, Pair):
            if level == 3:
                # We're going to explode the Pair pair.left at level 4
                # print(f"    Exploding left: {pair=}, {left=}, {right=}")
                pair_add(pair.right, pair.left.right, end="left")
                pair_add(left, pair.left.left, end="right")
                pair.left = Number(0)
                changed = True
            else:
                explode(pair.left, left, pair.right, level + 1)

        if (not changed) and isinstance(pair.right, Pair):
            if level == 3:
                # We're going to explode the Pair pair.right at level 4
                # print(f"    Exploding right: {pair=}, {left=}, {right=}")
                pair_add(pair.left, pair.right.left, end="right")
                pair_add(right, pair.right.right, end="left")
                pair.right = Number(0)
                changed = True
            else:
                explode(pair.right, pair.left, right, level + 1)

    def split(pair):
        nonlocal changed

        def split_number(number):
            nonlocal changed
            if (not changed) and (number >= Number(10)):
                changed = True
                return Pair(*number.split())
            else:
                return number
            
        if isinstance(pair.left, Number):
            pair.left = split_number(pair.left)
        else:
            split(pair.left)

        if isinstance(pair.right, Number):
            pair.right = split_number(pair.right)
        else:
            split(pair.right)

    
    # print(pair)
    while True:
        changed = False
        explode(pair)
        if changed:
            # print("After explode:", pair)
            continue
        split(pair)
        if changed:
            # print("After split:", pair)
            continue
        break

    return pair

In [ ]:
# Test the magnitude calculation against given examples
assert Pair.from_string("[[1,2],[[3,4],5]]").magnitude == 143
assert Pair.from_string("[[[[0,7],4],[[7,8],[6,0]]],[8,1]]").magnitude == 1384
assert Pair.from_string("[[[[1,1],[2,2]],[3,3]],[4,4]]").magnitude == 445
assert Pair.from_string("[[[[3,0],[5,3]],[4,4]],[5,5]]").magnitude == 791
assert Pair.from_string("[[[[5,0],[7,4]],[5,5]],[6,6]]").magnitude == 1137
assert Pair.from_string("[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]").magnitude == 3488

# Part 1

In [ ]:
def process_file(filename):
    pair = None
    with open(filename) as file:
        for line in file:
            if pair is None:
                pair = Pair.from_string(line)
            else:
                pair += Pair.from_string(line)
                pair = explode_and_split(pair)
    return pair

In [ ]:
pair = process_file("day18_example.input")
print(pair)
assert pair.magnitude == 4140

In [ ]:
pair = process_file("day18.input")
print(pair)
print(pair.magnitude)

# Part 2

In [ ]:
snailfish_numbers = []
with open("day18.input") as file:
    for line in file:
        snailfish_numbers.append(Pair.from_string(line))

In [ ]:
import itertools

magnitudes = []
for p1, p2 in itertools.permutations(snailfish_numbers, 2):
    magnitudes.append(explode_and_split(p1 + p2).magnitude)

max(magnitudes)